In [3]:
import sqlite3, csv
import pandas as pd

In [4]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [5]:
con = sqlite3.connect("new.db")
cur=con.cursor()



In [6]:
%sql sqlite:///new.db

In [8]:
df1=pd.read_csv(r"C:\Users\Ism\pythonProject\ChicagoCensusData.csv")
df2=pd.read_csv(r"C:\Users\Ism\pythonProject\ChicagoCrimeData.csv")
df3=pd.read_csv(r"C:\Users\Ism\pythonProject\ChicagoPublicSchools.csv")
df1.to_sql("ChicagoCensusData", con, if_exists='replace', index=False,method="multi")
df2.to_sql("ChicagoCrimeData", con, if_exists='replace', index=False,method="multi")
df3.to_sql("ChicagoPublicSchools", con, if_exists='replace', index=False,method="multi")

566

### SQLite: System Tables
##### to Know what all tables contain
#### https://www.techonthenet.com/sqlite/sys_tables/index.php 
I am using this link as a reference

In [25]:
%sql select * from sqlite_master

 * sqlite:///new.db
Done.


type,name,tbl_name,rootpage,sql
table,ChicagoCensusData,ChicagoCensusData,2,"CREATE TABLE ""ChicagoCensusData"" (""COMMUNITY_AREA_NUMBER"" REAL, ""COMMUNITY_AREA_NAME"" TEXT, ""PERCENT_OF_HOUSING_CROWDED"" REAL, ""PERCENT_HOUSEHOLDS_BELOW_POVERTY"" REAL, ""PERCENT_AGED_16__UNEMPLOYED"" REAL, ""PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA"" REAL, ""PERCENT_AGED_UNDER_18_OR_OVER_64"" REAL, ""PER_CAPITA_INCOME"" INTEGER, ""HARDSHIP_INDEX"" REAL)"
table,ChicagoCrimeData,ChicagoCrimeData,5,"CREATE TABLE ""ChicagoCrimeData"" (""ID"" INTEGER, ""CASE_NUMBER"" TEXT, ""DATE"" TEXT, ""BLOCK"" TEXT, ""IUCR"" TEXT, ""PRIMARY_TYPE"" TEXT, ""DESCRIPTION"" TEXT, ""LOCATION_DESCRIPTION"" TEXT, ""ARREST"" INTEGER, ""DOMESTIC"" INTEGER, ""BEAT"" INTEGER, ""DISTRICT"" INTEGER, ""WARD"" REAL, ""COMMUNITY_AREA_NUMBER"" REAL, ""FBICODE"" TEXT, ""X_COORDINATE"" REAL, ""Y_COORDINATE"" REAL, ""YEAR"" INTEGER, ""LATITUDE"" REAL, ""LONGITUDE"" REAL, ""LOCATION"" TEXT)"
table,ChicagoPublicSchools,ChicagoPublicSchools,29,"CREATE TABLE ""ChicagoPublicSchools"" (""School_ID"" INTEGER, ""NAME_OF_SCHOOL"" TEXT, ""ElementaryMiddleorHighSchool"" TEXT, ""Street_Address"" TEXT, ""City"" TEXT, ""State"" TEXT, ""ZIP_Code"" INTEGER, ""Phone_Number"" TEXT, ""Link"" TEXT, ""Network_Manager"" TEXT, ""Collaborative_Name"" TEXT, ""Adequate_Yearly_Progress_Made_"" TEXT, ""Track_Schedule"" TEXT, ""CPS_Performance_Policy_Status"" TEXT, ""CPS_Performance_Policy_Level"" TEXT, ""HEALTHY_SCHOOL_CERTIFIED"" TEXT, ""Safety_Icon"" TEXT, ""SAFETY_SCORE"" REAL, ""Family_Involvement_Icon"" TEXT, ""Family_Involvement_Score"" TEXT, ""Environment_Icon"" TEXT, ""Environment_Score"" REAL, ""Instruction_Icon"" TEXT, ""Instruction_Score"" REAL, ""Leaders_Icon"" TEXT, ""Leaders_Score"" TEXT, ""Teachers_Icon"" TEXT, ""Teachers_Score"" TEXT, ""Parent_Engagement_Icon"" TEXT, ""Parent_Engagement_Score"" TEXT, ""Parent_Environment_Icon"" TEXT, ""Parent_Environment_Score"" TEXT, ""AVERAGE_STUDENT_ATTENDANCE"" TEXT, ""Rate_of_Misconducts__per_100_students_"" REAL, ""Average_Teacher_Attendance"" TEXT, ""Individualized_Education_Program_Compliance_Rate"" TEXT, ""Pk_2_Literacy__"" TEXT, ""Pk_2_Math__"" TEXT, ""Gr3_5_Grade_Level_Math__"" TEXT, ""Gr3_5_Grade_Level_Read__"" TEXT, ""Gr3_5_Keep_Pace_Read__"" TEXT, ""Gr3_5_Keep_Pace_Math__"" TEXT, ""Gr6_8_Grade_Level_Math__"" TEXT, ""Gr6_8_Grade_Level_Read__"" TEXT, ""Gr6_8_Keep_Pace_Math_"" TEXT, ""Gr6_8_Keep_Pace_Read__"" TEXT, ""Gr_8_Explore_Math__"" TEXT, ""Gr_8_Explore_Read__"" TEXT, ""ISAT_Exceeding_Math__"" REAL, ""ISAT_Exceeding_Reading__"" REAL, ""ISAT_Value_Add_Math"" REAL, ""ISAT_Value_Add_Read"" REAL, ""ISAT_Value_Add_Color_Math"" TEXT, ""ISAT_Value_Add_Color_Read"" TEXT, ""Students_Taking__Algebra__"" TEXT, ""Students_Passing__Algebra__"" TEXT, ""9thGradeEXPLORE(2009)"" TEXT, ""9thGradeEXPLORE(2010)"" TEXT, ""10thGradePLAN(2009)"" TEXT, ""10thGradePLAN(2010)"" TEXT, ""Net_Change_EXPLORE_and_PLAN"" TEXT, ""11thGradeAverageACT(2011)"" TEXT, ""Net_Change_PLAN_and_ACT"" TEXT, ""College_Eligibility__"" TEXT, ""Graduation_Rate__"" TEXT, ""College_Enrollment_Rate__"" TEXT, ""COLLEGE_ENROLLMENT"" INTEGER, ""General_Services_Route"" INTEGER, ""Freshman_on_Track_Rate__"" TEXT, ""X_COORDINATE"" REAL, ""Y_COORDINATE"" REAL, ""Latitude"" REAL, ""Longitude"" REAL, ""COMMUNITY_AREA_NUMBER"" INTEGER, ""COMMUNITY_AREA_NAME"" TEXT, ""Ward"" INTEGER, ""Police_District"" INTEGER, ""Location"" TEXT)"


In [24]:
%%sql select * from sqlite_master
where tbl_name ="ChicagoCrimeData"

 * sqlite:///new.db
Done.


type,name,tbl_name,rootpage,sql
table,ChicagoCrimeData,ChicagoCrimeData,5,"CREATE TABLE ""ChicagoCrimeData"" (""ID"" INTEGER, ""CASE_NUMBER"" TEXT, ""DATE"" TEXT, ""BLOCK"" TEXT, ""IUCR"" TEXT, ""PRIMARY_TYPE"" TEXT, ""DESCRIPTION"" TEXT, ""LOCATION_DESCRIPTION"" TEXT, ""ARREST"" INTEGER, ""DOMESTIC"" INTEGER, ""BEAT"" INTEGER, ""DISTRICT"" INTEGER, ""WARD"" REAL, ""COMMUNITY_AREA_NUMBER"" REAL, ""FBICODE"" TEXT, ""X_COORDINATE"" REAL, ""Y_COORDINATE"" REAL, ""YEAR"" INTEGER, ""LATITUDE"" REAL, ""LONGITUDE"" REAL, ""LOCATION"" TEXT)"


### the total number of crimes recorded in the CRIME table

In [26]:
%sql select count(*) from ChicagoCrimeData

 * sqlite:///new.db
Done.


count(*)
533


### List community areas with per capita income less than 11000.

In [29]:
%sql select COMMUNITY_AREA_NAME from ChicagoCensusData where PER_CAPITA_INCOME <11000

 * sqlite:///new.db
Done.


COMMUNITY_AREA_NAME
West Garfield Park
South Lawndale
Fuller Park
Riverdale


### List all case numbers for crimes involving minors?

In [43]:
%sql select CASE_NUMBER from ChicagoCrimeData WHERE DESCRIPTION LIKE '%MINOR%';

 * sqlite:///new.db
Done.


CASE_NUMBER
HL266884
HK238408


### List all kidnapping crimes involving a child?(children are not considered minors for the purposes of crime analysis)

In [50]:
%sql select *	 from ChicagoCrimeData where PRIMARY_TYPE like 'KIDNAPPING'

 * sqlite:///new.db
Done.


ID,CASE_NUMBER,DATE,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,0,0,1533,15,29.0,25.0,20,1143050.0,1897546.0,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### What kind of crimes were recorded at schools?

In [55]:
%sql select distinct(PRIMARY_TYPE) from ChicagoCrimeData where LOCATION_DESCRIPTION like '%SCHOOL%'

 * sqlite:///new.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


### List the average safety score for all types of schools

In [59]:
%sql select ElementaryMiddleorHighSchool ,AVG(SAFETY_SCORE)from ChicagoPublicSchools group by ElementaryMiddleorHighSchool

 * sqlite:///new.db
Done.


ElementaryMiddleorHighSchool,AVG(SAFETY_SCORE)
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


### List 5 community areas with highest % of households below poverty line.

In [69]:
%%sql select COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME	,PERCENT_HOUSEHOLDS_BELOW_POVERTY from ChicagoCensusData 
order by PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC LIMIT 5

 * sqlite:///new.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
54.0,Riverdale,56.5
37.0,Fuller Park,51.2
68.0,Englewood,46.6
29.0,North Lawndale,43.1
27.0,East Garfield Park,42.4


### Which community area(number) is most crime prone

In [89]:
%%sql
SELECT COMMUNITY_AREA_NUMBER ,COUNT(COMMUNITY_AREA_NUMBER) AS FREQUENCY
FROM CHICAGOCRIMEDATA 
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(COMMUNITY_AREA_NUMBER) DESC
LIMIT 1

 * sqlite:///new.db
Done.


COMMUNITY_AREA_NUMBER,FREQUENCY
25.0,43


### Use a sub-query to find the name of the community area with highest hardship index.

In [93]:
%sql select COMMUNITY_AREA_NAME from  (select * from ChicagoCensusData order by HARDSHIP_INDEX DESC limit 1)

 * sqlite:///new.db
Done.


COMMUNITY_AREA_NAME
Riverdale


### sub-query to determine the Community Area Name with most number of crimes?

In [110]:
%%sql
with T1 as (select COMMUNITY_AREA_NUMBER , count(COMMUNITY_AREA_NUMBER) 
from CHICAGOCRIMEDATA
    group by COMMUNITY_AREA_NUMBER 
    order by count(COMMUNITY_AREA_NUMBER) DESC)
select COMMUNITY_AREA_NAME from ChicagoCensusData 
where(COMMUNITY_AREA_NUMBER like(select COMMUNITY_AREA_NUMBER from t1 limit 1)) 

 * sqlite:///new.db
Done.


COMMUNITY_AREA_NAME
Austin
